In [1]:
import os 
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir : Path
    train_data_path : Path
    test_data_path : Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str 

In [3]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, param_filepath= PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml (param_filepath)
        self.schema = read_yaml (schema_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_model_training_config(self): 
        config= self.config.model_trainer
        param = self.params.ElasticNet
        schema =self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        model_training_config= ModelTrainingConfig(
                root_dir= config.root_dir,
                train_data_path= config.train_data_path,
                test_data_path=  config.test_data_path,
                model_name= config.model_name,
                alpha= param.alpha,
                l1_ratio= param.l1_ratio,
                target_column= schema.name
        )
        return model_training_config 

In [ ]:
import pandas as pd 
import os 
from mlproject import logger 
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer(): 
    def __init__(self, config: ModelTrainingConfig):
        self.config = config 

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha= self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

[2024-12-30 04:25:48,423 - utils - INFO - NumExpr defaulting to 8 threads.]


In [6]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_training_config()
    model_trainer = ModelTrainer (config=model_trainer_config)
    model_trainer.train()
except Exception as e: 
    raise e

[2024-12-30 04:25:48,982 - common - INFO - yaml file: config\config.yaml loadded successfully]
[2024-12-30 04:25:48,984 - common - INFO - yaml file: params.yaml loadded successfully]
[2024-12-30 04:25:48,988 - common - INFO - yaml file: schema.yaml loadded successfully]
[2024-12-30 04:25:48,989 - common - INFO - created directory at: artifacts]
[2024-12-30 04:25:48,990 - common - INFO - created directory at: artifacts/mdoel_training]
